In [126]:
!nvidia-smi

Thu Sep 12 11:13:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8               4W /  30W |      8MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [127]:
import tensorflow as tf
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves,compare_historys

In [128]:
unzip_data('nlp-getting-started.zip')

In [129]:
import pandas as pd

In [130]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()
train_df["text"][1]


'Forest fire near La Ronge Sask. Canada'

In [131]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)

In [132]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [133]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [134]:
len(train_df),len(test_df)

(7613, 3263)

In [135]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target:{target}","(real disaster)" if target>0 else "(not rela disaster)")
    print(f"Text :{text}")
    print('\n')
    

Target:1 (real disaster)
Text :Today (August 6th) is the 70th anniversary of A-Bomb 'Little Boy' been dropped on Hiroshima.70000 killed outright as the city was flattened


Target:1 (real disaster)
Text :ÛÏRichmond Coaches were devastated to hear of the death of their second driver Mr Chance who was sittingÛ_: Jam... http://t.co/y5Yhbb0hkf


Target:1 (real disaster)
Text :Legionnaires' Disease: What's Being Done to Stop Deadly Outbreak: The worst-ever outbreak of Legio... http://t.co/0ubG9wFyge #losangeles


Target:1 (real disaster)
Text :CTD arrest three vital criminals from Orangi: KARACHI: Counter-Terrorism Department (CTD) of the Sindh Police ... http://t.co/Le4brduau9


Target:1 (real disaster)
Text :when a monster truck racer catches on fire at the fair




In [136]:
from sklearn.model_selection import train_test_split

train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size = 0.1,
                                                                       random_state=42)


In [137]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [138]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [139]:
from tensorflow.keras.layers import TextVectorization

In [140]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [141]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    standardize="lower_and_strip_punctuation",
    split = "whitespace",
    ngrams= None,
    output_mode = 'int',
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)


In [142]:
text_vectorizer.adapt(train_sentences)

In [143]:
sample_sentence = "hello world"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1400,  107,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [144]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
text_vectorizer(random_sentence)

@emaaalay thank you. ?? now I don't have a city wide curfew. ????


<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([   1,  449,   12,   48,    8,   63,   24,    3,  182, 4248,  378,
          0,    0,    0,    0])>

In [145]:
words_in_vocabulary =  text_vectorizer.get_vocabulary()
top_5_words = words_in_vocabulary[:5]
bottom_5_words = words_in_vocabulary[-5:]
len(words_in_vocabulary),top_5_words,bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [146]:
# from tensorflow.keras import layers

tf.random.set_seed(42)
from tensorflow.keras import layers

embeddings = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embeddings

/home/rasheek/Desktop/Github/NLP/NLP_fundamentals/tf/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding_1, built=False>

In [147]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
print("\n")
print("embedded version")
embeddings(text_vectorizer([random_sentence]))

@Snazzychipz OMG... WHAT IS THE SUB WEAPON


embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 3.6066536e-02,  9.4766840e-03, -8.3900988e-05, ...,
         -4.6096217e-02,  1.6818214e-02, -4.9287498e-02],
        [ 2.5978807e-02,  3.2574307e-02, -2.5332797e-02, ...,
          4.8150767e-02,  3.5608839e-02, -1.5054025e-02],
        [ 3.6931325e-02, -2.2440804e-02, -2.9876292e-02, ...,
          2.4503138e-02,  4.2801116e-02, -2.9451836e-02],
        ...,
        [-3.9189767e-02, -4.3304324e-02, -6.4241886e-04, ...,
         -5.7606697e-03,  1.8675353e-02, -1.8110514e-02],
        [-3.9189767e-02, -4.3304324e-02, -6.4241886e-04, ...,
         -5.7606697e-03,  1.8675353e-02, -1.8110514e-02],
        [-3.9189767e-02, -4.3304324e-02, -6.4241886e-04, ...,
         -5.7606697e-03,  1.8675353e-02, -1.8110514e-02]]], dtype=float32)>

In [148]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB()),
])

model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [149]:
baseline_score=model_0.score(val_sentences,val_labels)
baseline_score

0.7926509186351706

In [150]:
baseline_preds=model_0.predict(val_sentences)

In [151]:
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [152]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

In [153]:
def calculate_results(y_true,y_pred):
    model_accuaracy = accuracy_score(y_true,y_pred) *100
    model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true,y_pred,average="weighted")
    model_results={
        "accuracy":model_accuaracy,
        "precision":model_precision,
        "recall":model_recall,
        "f1":model_f1
    }
    return model_results

In [154]:
baseline_results=calculate_results(val_labels,baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [155]:
from helper_functions import create_tensorboard_callback

In [156]:
SAVE_DIR='model_logs'

In [157]:
from tensorflow.keras import  layers
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")

In [158]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_3      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [159]:
model_1.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [160]:
model_1.history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20240912-111340
Epoch 1/5


215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.6341 - loss: 0.6490 - val_accuracy: 0.7638 - val_loss: 0.5333
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8093 - loss: 0.4656 - val_accuracy: 0.7900 - val_loss: 0.4735
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8527 - loss: 0.3617 - val_accuracy: 0.7966 - val_loss: 0.4614
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8873 - loss: 0.2954 - val_accuracy: 0.7887 - val_loss: 0.4676
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9075 - loss: 0.2467 - val_accuracy: 0.7822 - val_loss: 0.4832


In [161]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [162]:
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7695 - loss: 0.5149


[0.48323386907577515, 0.7821522355079651]

In [163]:
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


(762, 1)

In [164]:
model_1_pred_probs[0]

array([0.3369913], dtype=float32)

In [165]:
model_1_pred_probs

array([[3.36991310e-01],
       [7.34024525e-01],
       [9.97574091e-01],
       [1.88334242e-01],
       [1.03878140e-01],
       [9.41557109e-01],
       [8.86982739e-01],
       [9.94287610e-01],
       [9.62175429e-01],
       [3.89513403e-01],
       [1.18733764e-01],
       [7.18412817e-01],
       [3.77492942e-02],
       [2.47215092e-01],
       [4.55935858e-03],
       [1.51755691e-01],
       [2.17685066e-02],
       [7.59515017e-02],
       [2.67087579e-01],
       [5.31628609e-01],
       [8.98761272e-01],
       [4.28090654e-02],
       [3.66759896e-01],
       [1.06901385e-01],
       [9.44217622e-01],
       [9.98816550e-01],
       [4.39224839e-02],
       [6.29978105e-02],
       [2.41327807e-02],
       [1.84399903e-01],
       [6.05541468e-01],
       [3.32586139e-01],
       [5.21996021e-01],
       [1.56354606e-01],
       [4.27458912e-01],
       [6.39995411e-02],
       [9.96223450e-01],
       [2.14407921e-01],
       [3.45602781e-02],
       [9.98741031e-01],


In [166]:
model_1_pred_probs[:10] 

array([[0.3369913 ],
       [0.7340245 ],
       [0.9975741 ],
       [0.18833424],
       [0.10387814],
       [0.9415571 ],
       [0.88698274],
       [0.9942876 ],
       [0.9621754 ],
       [0.3895134 ]], dtype=float32)

In [167]:
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))

In [168]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [169]:
model_1_results = calculate_results(val_labels,model_1_preds)

In [170]:
model_1_results

{'accuracy': 78.21522309711287,
 'precision': 0.7849731976577976,
 'recall': 0.7821522309711286,
 'f1': 0.7797935460636591}

In [171]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [172]:
import numpy as np 
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [173]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [174]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_3      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [175]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]

In [176]:
embed_weights

array([[-0.02935853, -0.04213534, -0.0051817 , ..., -0.00036777,
         0.00797984, -0.01514507],
       [ 0.05743232, -0.00693309, -0.05185055, ...,  0.05595633,
        -0.01801711, -0.00992973],
       [ 0.0326535 ,  0.01988475,  0.00592827, ..., -0.01490699,
        -0.03524092, -0.01868843],
       ...,
       [-0.04538136, -0.04546374, -0.02472192, ...,  0.01657373,
         0.03870112,  0.04251558],
       [ 0.05804038,  0.0020533 , -0.08560403, ...,  0.07715129,
        -0.04442613, -0.08638147],
       [ 0.04104766,  0.07099049, -0.06452668, ...,  0.04843402,
        -0.01841608, -0.07279117]], dtype=float32)

In [177]:
embed_weights.shape

(10000, 128)

In [178]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [179]:
from tensorflow.keras import  layers

In [180]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
# x=layers.LSTM(64,return_sequences=True)(x)
x=layers.LSTM(64)(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_2=tf.keras.Model(inputs,outputs,name="model_2_LTSM")

 

In [181]:
model_2.summary()

Model: "model_2_LTSM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [182]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [183]:
model_2.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LTSM")])

Saving TensorBoard log files to: model_logs/model_2_LTSM/20240912-111401
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.9175 - loss: 0.2891 - val_accuracy: 0.7769 - val_loss: 0.5404
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9361 - loss: 0.1692 - val_accuracy: 0.7782 - val_loss: 0.5967
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9478 - loss: 0.1382 - val_accuracy: 0.7717 - val_loss: 0.6859
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9581 - loss: 0.1139 - val_accuracy: 0.7690 - val_loss: 0.7747
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9669 - loss: 0.0925 - val_accuracy: 0.7638 - val_loss: 0.8806


In [184]:
model_2_pred_probs = model_2.predict(val_sentences)

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


In [185]:
model_1_pred_probs[:10]

array([[0.3369913 ],
       [0.7340245 ],
       [0.9975741 ],
       [0.18833424],
       [0.10387814],
       [0.9415571 ],
       [0.88698274],
       [0.9942876 ],
       [0.9621754 ],
       [0.3895134 ]], dtype=float32)

In [186]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [187]:
model_2_results = calculate_results(val_labels,model_2_preds)

In [188]:
model_2_results

{'accuracy': 76.37795275590551,
 'precision': 0.7654647714490235,
 'recall': 0.7637795275590551,
 'f1': 0.7615024149378589}

In [189]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [190]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
# x=layers.GRU(64,return_sequences=True)(x)
# x=layers.LSTM(64,return_sequences=True)(x)
# x=layers.GRU(64)(x)
# x=layers.GlobalAveragePooling1D()(x)
x=layers.GRU(64)(x)
x=layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_3=tf.keras.Model(inputs,outputs,name="model_3_GRU")




In [191]:
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_12 (GRU)                    │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,321,473 (5.04 MB)

 Trainable params: 1,321,473 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [192]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [193]:
model_3_history = model_3.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3_gru")])

Saving TensorBoard log files to: model_logs/model_3_gru/20240912-111429
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - accuracy: 0.8806 - loss: 0.2783 - val_accuracy: 0.7625 - val_loss: 0.7148
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9681 - loss: 0.0943 - val_accuracy: 0.7717 - val_loss: 0.8209
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9757 - loss: 0.0789 - val_accuracy: 0.7690 - val_loss: 0.9460
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9715 - loss: 0.0756 - val_accuracy: 0.7717 - val_loss: 1.0731
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9752 - loss: 0.0679 - val_accuracy: 0.7625 - val_loss: 1.1095


In [194]:
model_3_pred_probs=model_3.predict(val_sentences)
model_1_pred_probs

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step


array([[3.36991310e-01],
       [7.34024525e-01],
       [9.97574091e-01],
       [1.88334242e-01],
       [1.03878140e-01],
       [9.41557109e-01],
       [8.86982739e-01],
       [9.94287610e-01],
       [9.62175429e-01],
       [3.89513403e-01],
       [1.18733764e-01],
       [7.18412817e-01],
       [3.77492942e-02],
       [2.47215092e-01],
       [4.55935858e-03],
       [1.51755691e-01],
       [2.17685066e-02],
       [7.59515017e-02],
       [2.67087579e-01],
       [5.31628609e-01],
       [8.98761272e-01],
       [4.28090654e-02],
       [3.66759896e-01],
       [1.06901385e-01],
       [9.44217622e-01],
       [9.98816550e-01],
       [4.39224839e-02],
       [6.29978105e-02],
       [2.41327807e-02],
       [1.84399903e-01],
       [6.05541468e-01],
       [3.32586139e-01],
       [5.21996021e-01],
       [1.56354606e-01],
       [4.27458912e-01],
       [6.39995411e-02],
       [9.96223450e-01],
       [2.14407921e-01],
       [3.45602781e-02],
       [9.98741031e-01],


In [195]:
model_3_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [196]:
model_3_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [197]:
model_3_results=calculate_results(val_labels,model_3_preds)

In [198]:
model_3_results

{'accuracy': 78.21522309711287,
 'precision': 0.7849731976577976,
 'recall': 0.7821522309711286,
 'f1': 0.7797935460636591}

In [199]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
# x=layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
# x=layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

In [200]:
model_4.summary()

Model: "model_4_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,378,945 (5.26 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

In [201]:
model_4.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [202]:
model_4_history = model_4.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3_gru")])

Saving TensorBoard log files to: model_logs/model_3_gru/20240912-111500
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.9537 - loss: 0.1944 - val_accuracy: 0.7572 - val_loss: 1.0467
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9771 - loss: 0.0626 - val_accuracy: 0.7480 - val_loss: 1.2492
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9797 - loss: 0.0521 - val_accuracy: 0.7677 - val_loss: 1.0873
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9793 - loss: 0.0501 - val_accuracy: 0.7625 - val_loss: 1.1234
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9819 - loss: 0.0411 - val_accuracy: 0.7572 - val_loss: 1.1703


In [203]:
model_4_pred_probs=model_4.predict(val_sentences)

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


In [204]:
model_4_preds=tf.squeeze(tf.round(model_4_pred_probs))

In [205]:
model_4_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [206]:
model_4_results=calculate_results(val_labels,model_4_preds)

In [207]:
model_4_results

{'accuracy': 75.7217847769029,
 'precision': 0.7588431452858484,
 'recall': 0.7572178477690289,
 'f1': 0.7548067945124027}

In [208]:
embedding_test=embeddings(text_vectorizer(["This is a demo text "]))
conv_1d=layers.Conv1D(
    filters=32,
    kernel_size=4,
    activation="relu",
    padding="valid"
)
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPooling1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape , conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 12, 32]), TensorShape([1, 32]))

In [209]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.07104586,  0.01411878, -0.12570226, ...,  0.04970275,
         -0.07513319, -0.04949557],
        [-0.03420844, -0.01214469,  0.01346048, ..., -0.06237122,
         -0.01953427, -0.02775047],
        [-0.01851438,  0.0469859 , -0.01279371, ...,  0.04232066,
         -0.0289881 , -0.01233918],
        ...,
        [-0.08741628, -0.03199571,  0.01591121, ..., -0.05436816,
         -0.02905676, -0.02282028],
        [-0.08741628, -0.03199571,  0.01591121, ..., -0.05436816,
         -0.02905676, -0.02282028],
        [-0.08741628, -0.03199571,  0.01591121, ..., -0.05436816,
         -0.02905676, -0.02282028]]], dtype=float32)>

In [210]:
conv_1d_output

<tf.Tensor: shape=(1, 12, 32), dtype=float32, numpy=
array([[[0.        , 0.01503853, 0.06598306, 0.        , 0.        ,
         0.08113127, 0.        , 0.02692751, 0.03649754, 0.0232619 ,
         0.        , 0.        , 0.03088134, 0.        , 0.02830677,
         0.        , 0.        , 0.01690107, 0.03852435, 0.        ,
         0.03530596, 0.        , 0.05738579, 0.        , 0.        ,
         0.        , 0.06006774, 0.        , 0.        , 0.04769559,
         0.03011015, 0.05315686],
        [0.38669005, 0.        , 0.03343203, 0.        , 0.        ,
         0.19305927, 0.        , 0.        , 0.0108165 , 0.        ,
         0.4088841 , 0.        , 0.        , 0.        , 0.22577417,
         0.        , 0.        , 0.42523557, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.11671312, 0.        ,
         0.        , 0.        , 0.        , 0.18408018, 0.04266049,
         0.08453108, 0.1444122 ],
        [0.        , 0.        , 0.10127957, 0.    

In [211]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
x=layers.Conv1D(filters=32,
                kernel_size=5,
                strides=1,
                activation="relu",
                padding="valid")(x)
x=layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_5=tf.keras.Model(inputs,outputs,name="model_5_Conv1D")

In [212]:
model_5.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [213]:
model_5_history = model_4.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3_gru")])


Saving TensorBoard log files to: model_logs/model_3_gru/20240912-111533
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9836 - loss: 0.0370 - val_accuracy: 0.7612 - val_loss: 1.2506
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9816 - loss: 0.0437 - val_accuracy: 0.7585 - val_loss: 1.2701
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9820 - loss: 0.0393 - val_accuracy: 0.7612 - val_loss: 1.3970
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9826 - loss: 0.0376 - val_accuracy: 0.7690 - val_loss: 1.2351
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9836 - loss: 0.0361 - val_accuracy: 0.7454 - val_loss: 1.5618


In [214]:
model_5_pred_probs = model_5.predict(val_sentences)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [215]:
model_5_preds=tf.squeeze(tf.round(model_5_pred_probs))

In [216]:
model_5_results = calculate_results(val_labels,model_5_preds)

In [217]:
model_5_results

{'accuracy': 56.69291338582677,
 'precision': 0.5887107350896034,
 'recall': 0.5669291338582677,
 'f1': 0.48017415607172753}

In [222]:
import  tensorflow_hub as hub
embed=hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
embed_samples=embed([sample_sentence,"When you can the universal sentence encoder on a sentence ,  it turns it into numbers."])

print(embed_samples[0][:50])


2024-09-12 13:50:36.855802: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system memory.
2024-09-12 13:50:36.894493: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system memory.
2024-09-12 13:50:36.930176: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system memory.
2024-09-12 13:50:36.964884: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system memory.
2024-09-12 13:50:36.995900: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34133760 exceeds 10% of free system memory.


tf.Tensor(
[-0.02607422 -0.08460002 -0.02678661  0.05678423  0.06197045  0.03822602
  0.02011494  0.02740877  0.0869832   0.03079179  0.04104117  0.02551835
  0.00096512  0.05892798  0.04009536  0.04670895 -0.03312796  0.04070418
 -0.01199297 -0.04761707 -0.00829665  0.07054674  0.00484351  0.08098248
 -0.05701774 -0.06435516  0.06971473 -0.06988385  0.02646064 -0.02352202
  0.05523751  0.04913642  0.05698972 -0.06107194  0.03608509  0.02324465
  0.05601753 -0.01095515  0.04213083 -0.01769539  0.00712432  0.02047029
 -0.07101265 -0.08455775  0.04360583  0.04793973 -0.0901802  -0.06873439
  0.01279598  0.01624986], shape=(50,), dtype=float32)


In [224]:
from google.colab import files


ModuleNotFoundError: No module named 'google.colab'